In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from IPython.display import display
import pandas as pd
import time, json, random, re, datetime
import configparser as cp 

class InstagramScraper():
    def __init__(self):
        self.config = cp.ConfigParser()
        self.config.read('config.cfg')
        
        # Dummy account credentials
        self.username = self.config.get('core', 'username')
        self.password = self.config.get('core', 'password')
        
        # Change to whatever driver is preferred
        self.driver = webdriver.Chrome() 
        
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")
        
    def parse_json(self, page):
        # Create a bs4 object from page source
        soup = BeautifulSoup(page)
        
        # Grab interesting JSON only
        json_main = soup.find('script', string = re.compile("^window._sharedData") ).string.replace('window._sharedData = ', '')[:-1]
        
        # Parse the JSON into a dictionary
        parsed = json.loads(json_main)
        
        return(parsed)
    
    def remove_emojis(self, string):
        # Remove unwanted unicode emojies and special characters by forcing latin-1
        return(string.encode('latin-1', 'ignore').decode('utf-8'))
        
    def do_login(self):
        # Load page
        self.driver.get("https://www.instagram.com/accounts/login/")
        
        # Wait a bit for the page to load
        time.sleep(2)

        # Input the credentials
        self.driver.find_element_by_xpath("//input[@name='username']").send_keys(self.username)
        self.driver.find_element_by_xpath("//input[@name='password']").send_keys(self.password)

        # Send
        self.driver.find_element_by_xpath("//button[@type='submit']").click()
        
        # Wait a bit for the page to load after logging in
        time.sleep(2)
        
        return(True)
        
    # Scrape account metadata, image lists and followers
    def scrape_accounts(self, accounts = list(), do_profiles = True, do_posts = False, do_followers = False):
        # Prepare empty results lists
        profiles  = list()
        posts     = list()
        followers = list()
        
        # Cycle through target list and get metadata
        for one in accounts:
            self.driver.get('https://www.instagram.com/{0}/'.format(one))
            
            # check if the request returned 200
            if 'dialog-404' in self.driver.page_source:
                continue
            
            # wait a bit while the page loads
            time.sleep(2)
            
            # Get the profile metadata
            metadata = self.get_account_metadata(self.driver.page_source)
            profiles.append(metadata)
            
            if not metadata['is_private'] and do_posts:
                posts += self.get_posts(metadata)
            if not metadata['is_private'] and do_followers:
                followers.append(self.get_followers(metadata))
            
            # Randomized delay between requests
            time.sleep(random.randint(1,3))
        
        # Write this to a .csv file
        if do_profiles:
            display(pd.DataFrame(profiles))
            pd.DataFrame(profiles).to_csv('account_metadata/{0}.csv'.format(self.today.replace('-', '')), index=False)
        if not metadata['is_private'] and do_posts:
            df = pd.DataFrame(posts)
            df.drop_duplicates(inplace=True)
            display(df)
            df.to_csv('account_posts/{0}.csv'.format(self.today.replace('-', '')), index=False)
        if not metadata['is_private'] and do_followers:
            pd.DataFrame(followers).to_csv('account_followers/{0}.csv'.format(self.today.replace('-', '')), index=False)
            
    # Parse account metadata from html/json
    def get_account_metadata(self, page):
        # Parse the page JSON into a dictionary
        parsed = self.parse_json(page)
        
        metadata = {
            'date'      : self.today,
            'username'  : parsed['entry_data']['ProfilePage'][0]['graphql']['user']['username'],
            'full_name' : self.remove_emojis(parsed['entry_data']['ProfilePage'][0]['graphql']['user']['full_name']),
            'is_private': 1 if parsed['entry_data']['ProfilePage'][0]['graphql']['user']['is_private'] else 0,
            'is_business_account': 1 if parsed['entry_data']['ProfilePage'][0]['graphql']['user']['is_business_account'] else 0,
            'external_url': parsed['entry_data']['ProfilePage'][0]['graphql']['user']['external_url'],
            'biography' : self.remove_emojis(parsed['entry_data']['ProfilePage'][0]['graphql']['user']['biography'].replace('\n', ' ')),
            'business_category_name': parsed['entry_data']['ProfilePage'][0]['graphql']['user']['business_category_name'],
            'business_email': parsed['entry_data']['ProfilePage'][0]['graphql']['user']['business_email'],
            'following' : parsed['entry_data']['ProfilePage'][0]['graphql']['user']['edge_follow']['count'],
            'followers' : parsed['entry_data']['ProfilePage'][0]['graphql']['user']['edge_followed_by']['count'],
            'posts'     : parsed['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['count']
        }
        
        return(metadata)
        
    # Load all images through infinite scroll and grab each image's link and basic information
    def get_posts(self, metadata):
        # Since Instagram hides posts from the DOM tree as they disappear from view during scroll, we have to iterate over screens
        # The number of iterations = (total number of posts - the initial 12 images) / by the average of 10 images per scroll
        n = int((metadata['posts']-12)/10)
        posts = list()
        n = 2
        for i in range(n):
            # Scroll to the bottom of the page
            self.driver.find_element_by_tag_name('body').send_keys(Keys.END)
            
            # Find all posts visible in the DOM tree
            post_urls = self.driver.find_elements_by_xpath("//a[starts-with(@href,'/p/')]")
            
            for url in post_urls:
                # Add to list
                posts.append({
                    'date'    : self.today,
                    'username': metadata['username'],
                    'post_url': url.get_attribute('href')
                })
            
            # Randomized delay between requests
            time.sleep(random.randint(1,3))
        
        return(posts)
    
    def get_post_metadata(self, posts):
        user_posts = list()

        for one in posts:
            self.driver.get(one['post_url'])
            
            # Parse the page JSON into a dictionary
            parsed = self.parse_json(self.driver.page_source)

            metadata = {
                'date'      : self.today,
                'username'  : one['username'],
                'post_url'  : one['post_url'],
                'auto_generated_desc': parsed['entry_data']['PostPage'][0]['graphql']['shortcode_media']['accessibility_caption'],
                'is_video'  : 1 if parsed['entry_data']['PostPage'][0]['graphql']['shortcode_media']['is_video'] else 0,
                'caption'   : self.remove_emojis(parsed['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text']),
                'likes'     : parsed['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_preview_like']['count'],
                'comments'  : parsed['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_comment']['count'],
                'location'  : parsed['entry_data']['PostPage'][0]['graphql']['shortcode_media']['location']['name']
                # can add tagged users here, if needed
            }
            
            user_posts.append(metadata)
            
            # Randomized delay between requests
            time.sleep(random.randint(2,3))
        
        df = pd.DataFrame(user_posts)
        display(df)
        df.to_csv('account_posts/{0}.csv'.format(self.today.replace('-', '')), index=False)

    # Load all followers through infinite scroll and grab their usernames
    def get_followers(self, metadata):
        # Find the Followers link and click it
        self.driver.find_element_by_partial_link_text("follower").click()
        
        # Wait a bit for the modal to fully load
        time.sleep(3)
        
        # The first scroll only happens around this line, not at the bottom of the modal
        scroll = self.driver.find_element_by_xpath("//*[contains(text(), 'Suggestions For You')]")
        scroll.location_once_scrolled_into_view
        
        # The number of iterations = (total number of followers - the initial 12) / by the 12 followers per scroll
        n = int((metadata['followers']-12)/12)
        followers = list()
        
        # Look inside the modal window only
        modal = self.driver.find_element_by_xpath("//div[@role='dialog']")        
        n=2
        for i in range(n):
            # Then just find the last Follow button
            last_element = modal.find_elements_by_xpath("//div[@role='button']")[-1]
            last_element.location_once_scrolled_into_view

            # Delay between requests
            time.sleep(1)

        # Get all the links
        links = modal.find_elements_by_tag_name('a')
        lst = list()

        for i in links:
            lst.append(i.get_attribute('href'))

        # Remove duplicates, as every item has two '<a>' tags (redirect link and follow button)
        lst = list(dict.fromkeys(lst))

        followers += lst
        
        result = {
            'username': metadata['username'],
            'followers': followers
        }
        
        return(result)
        
    # Placeholder for retreiving comments
    def get_comments(self, post):
        pass